# PySeal Example 2
Firstly, let's import some modules that we will depend on in the next examples.

In [1]:
import seal
from seal import EncryptionParameters, \
    Evaluator, \
    Ciphertext, \
    Encryptor, \
    Decryptor, \
    IntegerEncoder, \
    KeyGenerator, \
    Plaintext, \
    SEALContext, \
    EvaluationKeys

In this example we explain what relinearization is, how to use it, and how
it affects noise budget consumption.

First we set the parameters, create a `SEALContext`, and generate the public
and secret keys. We use slightly larger parameters than be fore to be able
to do more homomorphic multiplications.

In [2]:
parms = EncryptionParameters()
parms.set_poly_modulus("1x^8192 + 1")

The default coefficient modulus consists of the following primes:

    0x7fffffffba0001,
    0x7fffffffaa0001,
    0x7fffffff7e0001,
    0x3fffffffd60001.
The total size is 219 bits.

In [3]:
def print_parameters(context):
    print("/ Encryption parameters:")
    print("| poly_modulus: " + context.poly_modulus().to_string())
    
    # Print the size of the true (product) coefficient modulus
    print("| coeff_modulus_size: " + (str)(context.total_coeff_modulus().significant_bit_count()) + " bits")

    print("| plain_modulus: " + (str)(context.plain_modulus().value()))
    print("| noise_standard_deviation: " + (str)(context.noise_standard_deviation()))

parms.set_coeff_modulus(seal.coeff_modulus_128(8192))
parms.set_plain_modulus(1 << 10)
context = SEALContext(parms)
print_parameters(context)
keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()

/ Encryption parameters:
| poly_modulus: 1x^8192 + 1
| coeff_modulus_size: 219 bits
| plain_modulus: 1024
| noise_standard_deviation: 3.19


We also set up an Encryptor, Evaluator, and Decryptor here. We will
encrypt polynomials directly in this example, so there is no need for
an encoder.

In [4]:
encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

There are actually two more types of keys in SEAL: `evaluation keys` and
`Galois keys`. Here we will discuss evaluation keys, and Galois keys will
be discussed later in example of batching.
In SEAL, a valid ciphertext consists of two or more polynomials with
coefficients integers modulo the product of the primes in coeff_modulus.
The current size of a ciphertext can be found using `Ciphertext::size()`.
A freshly encrypted ciphertext always has size 2.

In [5]:
plain1 = Plaintext("1x^2 + 2x^1 + 3")
encrypted = Ciphertext()
print("")
print("Encrypting " + plain1.to_string() + ": ")
encryptor.encrypt(plain1, encrypted)
print("Done")
print("Size of a fresh encryption: " + (str)(encrypted.size()))
print("Noise budget in fresh encryption: " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")


Encrypting 1x^2 + 2x^1 + 3: 
Done
Size of a fresh encryption: 2
Noise budget in fresh encryption: 197 bits


Homomorphic multiplication results in the output ciphertext growing in size.
More precisely, if the input ciphertexts have size M and N, then the output
ciphertext after homomorphic multiplication will have size M+N-1. In this
case we square encrypted twice to observe this growth (also observe noise
budget consumption).

In [6]:
evaluator.square(encrypted)
print("Size after squaring: " + (str)(encrypted.size()))
print("Noise budget after squaring: " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
plain2 = Plaintext()
decryptor.decrypt(encrypted, plain2)
print("Second power: " + plain2.to_string())
evaluator.square(encrypted);
print("Size after squaring: " + (str)(encrypted.size()))
print("Noise budget after squaring: " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")

Size after squaring: 3
Noise budget after squaring: 174 bits
Second power: 1x^4 + 4x^3 + Ax^2 + Cx^1 + 9
Size after squaring: 5
Noise budget after squaring: 140 bits


It does not matter that the size has grown -- decryption works as usual.
Observe from the print-out that the coefficients in the plaintext have
grown quite large. One more squaring would cause some of them to wrap
around plain_modulus (0x400), and as a result we would no longer obtain
the expected result as an integer-coefficient polynomial. We can fix this
problem to some extent by increasing plain_modulus. This would make sense,
since we still have plenty of noise budget left.

In [7]:
plain2 = Plaintext()
decryptor.decrypt(encrypted, plain2)
print("Fourth power: " + plain2.to_string())

Fourth power: 1x^8 + 8x^7 + 24x^6 + 68x^5 + D6x^4 + 138x^3 + 144x^2 + D8x^1 + 51


The problem here is that homomorphic operations on large ciphertexts are
computationally much more costly than on small ciphertexts. Specifically,
homomorphic multiplication on input ciphertexts of size M and N will require
O(M*N) polynomial multiplications to be performed, and an addition will
require O(M+N) additions. Relinearization reduces the size of the ciphertexts
after multiplication back to the initial size (2). Thus, relinearizing one
or both inputs before the next multiplication, or e.g. before serializing the
ciphertexts, can have a huge positive impact on performance.
Another problem is that the noise budget consumption in multiplication is
bigger when the input ciphertexts sizes are bigger. In a complicated
computation the contribution of the sizes to the noise budget consumption
can actually become the dominant term. We will point this out again below
once we get to our example.
Relinearization itself has both a computational cost and a noise budget cost.
These both depend on a parameter called `decomposition bit count`, which can
be any integer at least 1 `dbc_min()` and at most 60 `dbc_max()`. A large
decomposition bit count makes relinearization fast, but consumes more noise
budget. A small decomposition bit count can make relinearization slower, but
might not change the noise budget by any observable amount.
Relinearization requires a special type of key called `evaluation keys`.
These can be created by the KeyGenerator for any decomposition bit count.
To relinearize a ciphertext of size M >= 2 back to size 2, we actually need
M-2 evaluation keys. Attempting to relinearize a too large ciphertext with
too few evaluation keys will result in an exception being thrown.
We repeat our computation, but this time relinearize after both squarings.
Since our ciphertext never grows past size 3 (we relinearize after every
multiplication), it suffices to generate only one evaluation key.
First, we need to create evaluation keys. We use a decomposition bit count
of 16 here, which can be thought of as quite small.

In [8]:
ev_keys16 = EvaluationKeys()

This function generates one single evaluation key. Another overload takes
the number of keys to be generated as an argument, but one is all we need
in this example (see above).

In [9]:
keygen.generate_evaluation_keys(16, ev_keys16)
print("")
print("Encrypting " + plain1.to_string() + ": ")
encryptor.encrypt(plain1, encrypted)
print("Done")
print("Size of a fresh encryption: " + (str)(encrypted.size()))
print("Noise budget in fresh encryption: " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.square(encrypted)
print("Size after squaring: " + (str)(encrypted.size()))
print("Noise budget after squaring: " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.relinearize(encrypted, ev_keys16)
print("Size after relinearization: " + (str)(encrypted.size()))
print("Noise budget after relinearizing (dbs = " + (str)(ev_keys16.decomposition_bit_count()) + "): " +
      (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.square(encrypted)
print("Size after second squaring: " + (str)(encrypted.size()) + " bits")
print("Noise budget after second squaring: " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.relinearize(encrypted, ev_keys16)
print("Size after relinearization: " + (str)(encrypted.size()))
print("Noise budget after relinearizing (dbs = " + (str)(ev_keys16.decomposition_bit_count()) + "): " +
      (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
decryptor.decrypt(encrypted, plain2)
print("Fourth power: " + plain2.to_string())


Encrypting 1x^2 + 2x^1 + 3: 
Done
Size of a fresh encryption: 2
Noise budget in fresh encryption: 197 bits
Size after squaring: 3
Noise budget after squaring: 174 bits
Size after relinearization: 2
Noise budget after relinearizing (dbs = 16): 174 bits
Size after second squaring: 3 bits
Noise budget after second squaring: 146 bits
Size after relinearization: 2
Noise budget after relinearizing (dbs = 16): 146 bits
Fourth power: 1x^8 + 8x^7 + 24x^6 + 68x^5 + D6x^4 + 138x^3 + 144x^2 + D8x^1 + 51


Of course the result is still the same, but this time we actually
used less of our noise budget. This is not surprising for two reasons:
    - We used a very small decomposition bit count, which is why
      relinearization itself did not consume the noise budget by any
      observable amount;
    - Since our ciphertext sizes remain small throughout the two
      squarings, the noise budget consumption rate in multiplication
      remains as small as possible. Recall from above that operations
      on larger ciphertexts actually cause more noise growth.
To make matters even more clear, we repeat the computation a third time,
now using the largest possible decomposition bit count (60). We are not
measuring the time here, but relinearization with these evaluation keys
is significantly faster than with ev_keys16.

In [10]:
ev_keys60 = EvaluationKeys()
keygen.generate_evaluation_keys(seal.dbc_max(), ev_keys60)
print("")
print("Encrypting " + plain1.to_string() + ": ")
encryptor.encrypt(plain1, encrypted)
print("Done")
print("Size of a fresh encryption: " + (str)(encrypted.size()))
print("Noise budget in fresh encryption: " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.square(encrypted)
print("Size after squaring: " + (str)(encrypted.size()))
print("Noise budget after squaring: " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.relinearize(encrypted, ev_keys60)
print("Size after relinearization: " + (str)(encrypted.size()))
print("Noise budget after relinearizing (dbc = " + (str)(ev_keys60.decomposition_bit_count()) +
      "): " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.square(encrypted)
print("Size after second squaring: " + (str)(encrypted.size()))
print("Noise budget after second squaring: " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.relinearize(encrypted, ev_keys60)
print("Size after relinearization: " + (str)(encrypted.size()))
print("Noise budget after relinearizing (dbc = " + (str)(ev_keys60.decomposition_bit_count()) +
      "): " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
decryptor.decrypt(encrypted, plain2)
print("Fourth power: " + plain2.to_string())


Encrypting 1x^2 + 2x^1 + 3: 
Done
Size of a fresh encryption: 2
Noise budget in fresh encryption: 197 bits
Size after squaring: 3
Noise budget after squaring: 174 bits
Size after relinearization: 2
Noise budget after relinearizing (dbc = 60): 142 bits
Size after second squaring: 3
Noise budget after second squaring: 114 bits
Size after relinearization: 2
Noise budget after relinearizing (dbc = 60): 114 bits
Fourth power: 1x^8 + 8x^7 + 24x^6 + 68x^5 + D6x^4 + 138x^3 + 144x^2 + D8x^1 + 51


Observe from the print-out that we have now used significantly more of our
noise budget than in the two previous runs. This is again not surprising,
since the first relinearization chops off a huge part of the noise budget.
However, note that the second relinearization does not change the noise
budget by any observable amount. This is very important to understand when
optimal performance is desired: relinearization always drops the noise
budget from the maximum (freshly encrypted ciphertext) down to a fixed
amount depending on the encryption parameters and the decomposition bit
count. On the other hand, homomorphic multiplication always consumes the
noise budget from its current level. This is why the second relinearization
does not change the noise budget anymore: it is already consumed past the
fixed amount determinted by the decomposition bit count and the encryption
parameters.
We now perform a third squaring and observe an even further compounded
decrease in the noise budget. Again, relinearization does not consume the
noise budget at this point by any observable amount, even with the largest
possible decomposition bit count.

In [11]:
evaluator.square(encrypted)
print("Size after third squaring " + (str)(encrypted.size()))
print("Noise budget after third squaring: " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
evaluator.relinearize(encrypted, ev_keys60)
print("Size after relinearization: " + (str)(encrypted.size()))
print("Noise budget after relinearizing (dbc = " + (str)(ev_keys60.decomposition_bit_count()) +
      "): " + (str)(decryptor.invariant_noise_budget(encrypted)) + " bits")
decryptor.decrypt(encrypted, plain2)
print("Eighth power: " + plain2.to_string())

Size after third squaring 3
Noise budget after third squaring: 85 bits
Size after relinearization: 2
Noise budget after relinearizing (dbc = 60): 85 bits
Eighth power: 1x^16 + 10x^15 + 88x^14 + 310x^13 + 13Cx^12 + 110x^11 + 78x^10 + 390x^9 + 1A6x^8 + 2B0x^7 + 38x^6 + B0x^5 + 3FCx^4 + 30x^3 + 348x^2 + B0x^1 + 1A1


Observe from the print-out that the polynomial coefficients are no longer
correct as integers: they have been reduced modulo plain_modulus, and there
was no warning sign about this. It might be necessary to carefully analyze
the computation to make sure such overflow does not occur unexpectedly.
These experiments suggest that an optimal strategy might be to relinearize
first with evaluation keys with a small decomposition bit count, and later
with evaluation keys with a larger decomposition bit count (for performance)
when noise budget has already been consumed past the bound determined by the
larger decomposition bit count. For example, above the best strategy might
have been to use ev_keys16 in the first relinearization, and ev_keys60 in the
next two relinearizations for optimal noise budget consumption/performance
trade-off. Luckily, in most use-cases it is not so critical to squeeze out
every last bit of performance, especially when slightly larger parameters
are used.